# Deep Q-Learning 

For this assignment we will implement the Deep Q-Learning algorithm with Experience Replay as described in breakthrough paper __"Playing Atari with Deep Reinforcement Learning"__. We will train an agent to play the famous game of __Breakout__.

In [1]:
import sys
import gym
import torch
import pylab
import random
import numpy as np
from collections import deque
from datetime import datetime
from copy import deepcopy
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from utils import *
from agent import *
from model import *
from config import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Understanding the environment

In the following cell, we initialise our game of __Breakout__ and you can see how the environment looks like. For further documentation of the of the environment refer to https://gym.openai.com/envs. 

In [ ]:
env = gym.make('SpaceInvaders-v0')
#env.render()

In [ ]:
number_lives = find_max_lifes(env)
state_size = env.observation_space.shape
action_size = 6
rewards, episodes = [], []

## Creating a DQN Agent

Here we create a DQN Agent. This agent is defined in the __agent.py__. The corresponding neural network is defined in the __model.py__. 

__Evaluation Reward__ : The average reward received in the past 100 episodes/games.

__Frame__ : Number of frames processed in total.

__Memory Size__ : The current size of the replay memory.

In [ ]:
agent = Agent(action_size)
evaluation_reward = deque(maxlen=evaluation_reward_length)
frame = 0
memory_size = 0


### Main Training Loop

In [ ]:
for e in range(EPISODES):
    done = False
    score = 0

    history = np.zeros([5, 84, 84], dtype=np.uint8)
    step = 0
    d = False
    state = env.reset()
    life = number_lives

    get_init_state(history, state)

    while not done:
        step += 1
        frame += 1
        if render_breakout:
            env.render()

        # Select and perform an action
        action, value = agent.get_action(np.float32(history[:4, :, :]) / 255.)

        
        next_state, reward, done, info = env.step(action)

        frame_next_state = get_frame(next_state)
        history[4, :, :] = frame_next_state
        terminal_state = check_live(life, info['ale.lives'])

        life = info['ale.lives']
        #r = np.clip(reward, -1, 1)
        r = reward
        
        # Store the transition in memory 
        agent.memory.push(deepcopy(frame_next_state), action, r, terminal_state, value, 0, 0)
        # Start training after random sample generation
        if(frame % train_frame == 0):
            agent.train_policy_net(frame)
            # Update the target network
            agent.update_target_net()
        score += r
        history[:4, :, :] = history[1:, :, :]

        if frame % 50000 == 0:
            print('now time : ', datetime.now())
            rewards.append(np.mean(evaluation_reward))
            episodes.append(e)
            pylab.plot(episodes, rewards, 'b')
            pylab.savefig("./save_graph/breakout_dqn.png")

        if done:
            evaluation_reward.append(score)
            # every episode, plot the play time
            print("episode:", e, "  score:", score, "  memory length:",
                  len(agent.memory), "  epsilon:", agent.epsilon, "   steps:", step,
                  "    evaluation reward:", np.mean(evaluation_reward))

            # if the mean of scores of last 10 episode is bigger than 400
            # stop training
            if np.mean(evaluation_reward) > 40 and len(evaluation_reward) > 700:
                torch.save(agent.policy_net, "./save_model/breakout_dqn")
                sys.exit()

/home/sigai/anaconda3/envs/py36/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/model.py:45: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  probs = F.softmax(x[:,:self.action_size] - torch.max(x[:,:self.action_size],0)[0])


episode: 0   score: 185.0   memory length: 821   epsilon: 1.0    steps: 821     evaluation reward: 185.0
Training network


/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:241: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  pol_loss += pol_avg.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:242: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  vf_loss += value_loss.detach().cpu()[0]
/home/sigai/Documents/Projects/Object-Recognition/assignment5_materials/Assignment5_PPO/Assignment5/agent.py:243: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
  ent_total += ent.detach().cpu()[0]


Iteration 1: Policy loss: -13.282112. Value loss: 12.888132. Entropy: 1.790153.
Iteration 2: Policy loss: -13.259413. Value loss: 12.864014. Entropy: 1.790195.
Iteration 3: Policy loss: -13.290983. Value loss: 12.893475. Entropy: 1.790215.
episode: 1   score: 110.0   memory length: 1024   epsilon: 1.0    steps: 676     evaluation reward: 147.5
Training network
Iteration 4: Policy loss: -11.884912. Value loss: 11.576634. Entropy: 1.790292.
Iteration 5: Policy loss: -12.058539. Value loss: 11.745104. Entropy: 1.790237.
Iteration 6: Policy loss: -11.863825. Value loss: 11.551113. Entropy: 1.790263.
episode: 2   score: 165.0   memory length: 1024   epsilon: 1.0    steps: 676     evaluation reward: 153.33333333333334
episode: 3   score: 105.0   memory length: 1024   epsilon: 1.0    steps: 533     evaluation reward: 141.25
Training network
Iteration 7: Policy loss: -7.510808. Value loss: 7.236870. Entropy: 1.790373.
Iteration 8: Policy loss: -7.452139. Value loss: 7.178435. Entropy: 1.790372

episode: 29   score: 105.0   memory length: 1024   epsilon: 1.0    steps: 476     evaluation reward: 158.33333333333334
Training network
Iteration 61: Policy loss: -11.958920. Value loss: 11.627741. Entropy: 1.790206.
Iteration 62: Policy loss: -11.702659. Value loss: 11.371776. Entropy: 1.790171.
Iteration 63: Policy loss: -11.918467. Value loss: 11.584954. Entropy: 1.790172.
episode: 30   score: 50.0   memory length: 1024   epsilon: 1.0    steps: 419     evaluation reward: 154.83870967741936
Training network
Iteration 64: Policy loss: -12.631729. Value loss: 12.279160. Entropy: 1.790323.
Iteration 65: Policy loss: -12.527327. Value loss: 12.177687. Entropy: 1.790295.
Iteration 66: Policy loss: -12.613468. Value loss: 12.262482. Entropy: 1.790321.
episode: 31   score: 205.0   memory length: 1024   epsilon: 1.0    steps: 976     evaluation reward: 156.40625
Training network
Iteration 67: Policy loss: -11.393254. Value loss: 11.068705. Entropy: 1.790371.
Iteration 68: Policy loss: -11.2

...

episode: 664   score: 165.0   memory length: 1024   epsilon: 1.0    steps: 1094     evaluation reward: 138.05
Training network
Iteration 1345: Policy loss: -5.520286. Value loss: 5.967849. Entropy: 1.790428.
Iteration 1346: Policy loss: -5.535381. Value loss: 5.985423. Entropy: 1.790440.
Iteration 1347: Policy loss: -5.522268. Value loss: 5.971350. Entropy: 1.790423.
episode: 665   score: 335.0   memory length: 1024   epsilon: 1.0    steps: 806     evaluation reward: 140.5
Training network
Iteration 1348: Policy loss: -18.719652. Value loss: 18.490595. Entropy: 1.790364.
Iteration 1349: Policy loss: -18.810081. Value loss: 18.569818. Entropy: 1.790370.
Iteration 1350: Policy loss: -18.539978. Value loss: 18.301695. Entropy: 1.790400.
episode: 666   score: 135.0   memory length: 1024   epsilon: 1.0    steps: 671     evaluation reward: 140.65
episode: 667   score: 75.0   memory length: 1024   epsilon: 1.0    steps: 533     evaluation reward: 140.2
Training network
Iteration 1351: Policy 

Iteration 1404: Policy loss: -8.955646. Value loss: 8.973459. Entropy: 1.790337.
episode: 692   score: 105.0   memory length: 1024   epsilon: 1.0    steps: 622     evaluation reward: 147.05
episode: 693   score: 80.0   memory length: 1024   epsilon: 1.0    steps: 739     evaluation reward: 143.0
Training network
Iteration 1405: Policy loss: -6.645695. Value loss: 7.048760. Entropy: 1.790682.
Iteration 1406: Policy loss: -6.548018. Value loss: 6.954194. Entropy: 1.790653.
Iteration 1407: Policy loss: -6.453750. Value loss: 6.864163. Entropy: 1.790635.
episode: 694   score: 105.0   memory length: 1024   epsilon: 1.0    steps: 593     evaluation reward: 142.4
episode: 695   score: 90.0   memory length: 1024   epsilon: 1.0    steps: 556     evaluation reward: 141.75
Training network
Iteration 1408: Policy loss: -8.863056. Value loss: 8.936185. Entropy: 1.790668.
Iteration 1409: Policy loss: -8.640089. Value loss: 8.723355. Entropy: 1.790679.
Iteration 1410: Policy loss: -8.679076. Value lo

episode: 721   score: 110.0   memory length: 1024   epsilon: 1.0    steps: 630     evaluation reward: 147.2
episode: 722   score: 45.0   memory length: 1024   epsilon: 1.0    steps: 382     evaluation reward: 146.85
Training network
Iteration 1462: Policy loss: -7.250010. Value loss: 7.356271. Entropy: 1.790648.
Iteration 1463: Policy loss: -7.244108. Value loss: 7.350345. Entropy: 1.790645.
Iteration 1464: Policy loss: -7.183852. Value loss: 7.295191. Entropy: 1.790651.
episode: 723   score: 65.0   memory length: 1024   epsilon: 1.0    steps: 416     evaluation reward: 143.65
now time :  2018-12-19 13:35:29.101346
episode: 724   score: 155.0   memory length: 1024   epsilon: 1.0    steps: 890     evaluation reward: 144.5
Training network
Iteration 1465: Policy loss: -8.102412. Value loss: 8.287089. Entropy: 1.790618.
Iteration 1466: Policy loss: -8.208789. Value loss: 8.384427. Entropy: 1.790605.
Iteration 1467: Policy loss: -8.122915. Value loss: 8.301895. Entropy: 1.790610.
episode: 

Iteration 1517: Policy loss: -10.066642. Value loss: 10.015295. Entropy: 1.790622.
Iteration 1518: Policy loss: -9.987080. Value loss: 9.946028. Entropy: 1.790677.
episode: 752   score: 130.0   memory length: 1024   epsilon: 1.0    steps: 406     evaluation reward: 146.0
episode: 753   score: 100.0   memory length: 1024   epsilon: 1.0    steps: 850     evaluation reward: 146.2
Training network
Iteration 1519: Policy loss: -5.543866. Value loss: 5.984920. Entropy: 1.790499.
Iteration 1520: Policy loss: -5.501387. Value loss: 5.939048. Entropy: 1.790498.
Iteration 1521: Policy loss: -5.397815. Value loss: 5.848903. Entropy: 1.790493.
episode: 754   score: 130.0   memory length: 1024   epsilon: 1.0    steps: 632     evaluation reward: 147.4
Training network
Iteration 1522: Policy loss: -8.617120. Value loss: 9.008932. Entropy: 1.790521.
Iteration 1523: Policy loss: -8.607680. Value loss: 8.993595. Entropy: 1.790482.
Iteration 1524: Policy loss: -8.681946. Value loss: 9.065740. Entropy: 1.

episode: 780   score: 20.0   memory length: 1024   epsilon: 1.0    steps: 409     evaluation reward: 142.75
Training network
Iteration 1576: Policy loss: -13.459755. Value loss: 13.743070. Entropy: 1.790712.
Iteration 1577: Policy loss: -13.242714. Value loss: 13.536271. Entropy: 1.790722.
Iteration 1578: Policy loss: -13.224059. Value loss: 13.513487. Entropy: 1.790715.
episode: 781   score: 225.0   memory length: 1024   epsilon: 1.0    steps: 814     evaluation reward: 143.45
episode: 782   score: 110.0   memory length: 1024   epsilon: 1.0    steps: 750     evaluation reward: 142.75
Training network
Iteration 1579: Policy loss: -6.319079. Value loss: 6.797122. Entropy: 1.790463.
Iteration 1580: Policy loss: -6.416790. Value loss: 6.889909. Entropy: 1.790490.
Iteration 1581: Policy loss: -6.238068. Value loss: 6.717836. Entropy: 1.790449.
episode: 783   score: 140.0   memory length: 1024   epsilon: 1.0    steps: 641     evaluation reward: 142.35
Training network
Iteration 1582: Policy

In [ ]:
torch.save(agent.policy_net, "./save_model/breakout_dqn")